In [ ]:
import pandas as pd
import numpy as np
import torch
import io
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time
from sklearn.metrics import f1_score


In [ ]:
%ls
prefix = '/data1/tjss/'
# prefix = '/content/drive/My Drive/Summer research /'
doc_level_df = pd.read_csv(prefix+'data/msgs_tec.csv')
word_level_df = pd.read_csv(prefix+'data/NRC_ht_emotion_sentiment.csv')

=0.0.7            Miniconda3-latest-Linux-x86_64.sh          test/
examples.desktop  nltk_data/                                 tut3-model.pt
jupyter           NRC_Emotion_Classification_DeepShap.ipynb
miniconda3/       PyTorch_Classification__New.ipynb


/home/tjss/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#EDA
print(doc_level_df.shape)
print(doc_level_df.emotion.unique())
doc_level_df = doc_level_df.dropna()
print(doc_level_df.shape)
doc_level_df['message'] = doc_level_df['message'].str.lower() 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
doc_level_df['emotion_code'] = lb_make.fit_transform(doc_level_df['emotion'])
print(doc_level_df.iloc[:10,:])
new_doc_level_df = doc_level_df[['message', 'emotion_code']]
print(new_doc_level_df.iloc[:10,:])
new_doc_level_df.head(10)

new_doc_level_df.loc[(new_doc_level_df.emotion_code!=3),'emotion_code']=0
new_doc_level_df.loc[(new_doc_level_df.emotion_code==3),'emotion_code']=1

new_doc_level_train, new_doc_level_test = train_test_split(new_doc_level_df, test_size=0.2, random_state=1)
new_doc_level_train, new_doc_level_val = train_test_split(new_doc_level_train, test_size=0.25, random_state=1)

(21051, 3)
['surprise' 'sadness' 'joy' 'disgust' 'fear' 'anger' nan]
(21049, 3)
           message_id                                            message  \
0  145353048817012736  thinks that @melbahughes had a great 50th birt...   
1  144279638024257536  como una expresiã³n tan simple, una sola oraci...   
2  140499585285111809  the moment when you get another follower and y...   
3  145207578270507009  be the greatest dancer of your life! practice ...   
4  139502146390470656  eww.. my moms starting to make her annual rum ...   
5  146042696899887106  if ur heart hurts all the time for tht person ...   
6  145492569609084928  i feel awful, and it's way too freaking early....   
7  145903955229151232  so chuffed for safc fans! bet me dar comes in ...   
8  142717613234069504  making art and viewing art are different at th...   
9  144183822873927680  soooo dooowwwn!! move on, get some sleep... me...   

    emotion  emotion_code  
0  surprise             5  
1   sadness             4  

/home/tjss/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
print(new_doc_level_train.emotion_code.value_counts())
print(new_doc_level_val.emotion_code.value_counts())
print(new_doc_level_test.emotion_code.value_counts())

0    7706
1    4923
Name: emotion_code, dtype: int64
0    2570
1    1640
Name: emotion_code, dtype: int64
0    2534
1    1676
Name: emotion_code, dtype: int64


In [ ]:
print(new_doc_level_train.shape)
print(new_doc_level_val.shape)
print(new_doc_level_test.shape)

(12629, 2)
(4210, 2)
(4210, 2)


In [ ]:
# 0-> not Joy
# 1-> Joy


# 0 Anger
# 1 Disgust
# 2 Fear
# 3 Joy
# 4 Sadness
# 5 Surprise

In [ ]:
new_doc_level_train.to_csv(prefix+"data/new_doc_level_train.csv")
new_doc_level_val.to_csv(prefix+"data/new_doc_level_val.csv")
new_doc_level_test.to_csv(prefix+"data/new_doc_level_test.csv")

In [ ]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data


In [ ]:
embeddings = load_vectors(prefix+"data/crawl-300d-2M.vec")

In [ ]:
print(type(embeddings))
print(len(embeddings))
for word in embeddings:
  embeddings[word] = [float(i) for i in embeddings[word]]
embeddings_keylist = list(embeddings)
embeddings_vectorlist = list(embeddings.values())
word_to_index={}
index_to_word={}
for i in range(len(embeddings_keylist)):
  word_to_index[embeddings_keylist[i]]=i
  index_to_word[i]=embeddings_keylist[i]

<class 'dict'>
1999995


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# from spacy.lang.en import English
# nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
# tokenizer = nlp.Defaults.create_tokenizer(nlp)



[nltk_data] Downloading package punkt to /home/tjss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# # Dummy dataloader
# test_df = pd.read_csv(prefix+"data/new_doc_level_val.csv")
# print(test_df.shape)
# test_df = test_df.iloc[:,1:]
# curr_tex = test_df.iloc[6,0]
# curr_emot = test_df.iloc[6,1]
# curr_embed = np.zeros((300))
# print("curr text", curr_tex)
# print(curr_embed)
# tot_cnt=0
# ign_cnt=0
# for word in word_tokenize(curr_tex):
#   print(word)
#   tot_cnt+=1
#   if word in embeddings.keys():
#     curr_embed = curr_embed+embeddings[word]
#   else:
#     ign_cnt+=1
# print("percentage lost", ign_cnt/tot_cnt*100, "%")
# print(curr_embed.shape)
# curr_embed = curr_embed
# embed_t = torch.from_numpy(curr_embed)
# print(embed_t)

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"

# kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
class FastTextVectorLoader(Dataset):
  def __init__(self, csv_file, embeddings):
    self.frame = pd.read_csv(csv_file)
    self.embeddings = embeddings
  def __len__(self):
    return len(self.frame)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    curr_tex = self.frame.iloc[idx,1]
    curr_emot = self.frame.iloc[idx,2]
    curr_embed = np.zeros((300))
    tot_cnt=0
    ign_cnt=0
    
    for word in word_tokenize(curr_tex):
      tot_cnt+=1
      if word in embeddings.keys():
        # print(word)
        curr_embed = curr_embed+embeddings[word]
      else:
        ign_cnt+=1
    # print("percentage lost", ign_cnt/tot_cnt*100, "%")
    curr_embed = curr_embed
    embed_t = torch.from_numpy(curr_embed)
    # embed_t = embed_t.to(torch.device("cuda:0"))
    # curr_emot = curr_emot.cuda()
    data = {'vector':embed_t, 'label':curr_emot}
    return data

In [ ]:
train_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_train.csv", embeddings)
val_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_val.csv", embeddings)
test_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_test.csv", embeddings)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64,shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=64,shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1)


In [ ]:
# testing dataloader
count = 0
for i,batch_data in enumerate(train_dataloader):
  count += 1
  print(batch_data['vector'])
  print(batch_data['vector'].size())
  print(batch_data['label'])
  # batch_data['vector'].cuda()
  # batch_data['vector'].to(device)
  print(batch_data['vector'].is_cuda)
  print(batch_data['label'].is_cuda)
  # print(text)
  # print(text_length)
  if count == 1:
    break


tensor([[-0.1529, -0.9986,  0.4191,  ..., -0.2421, -0.0856, -0.2003],
        [-2.3536, -1.4974, -0.9999,  ...,  0.3417, -3.1617, -2.3938],
        [ 1.0120, -1.2992, -1.6559,  ...,  0.3428, -3.6327, -0.1696],
        ...,
        [-1.4158,  0.5236, -1.1670,  ..., -0.2287, -1.9959, -1.1974],
        [-0.3894, -0.3990, -0.8730,  ...,  2.3104, -1.2938, -0.4660],
        [-0.4092, -1.1203,  0.9604,  ...,  0.0851, -0.7187,  0.2072]],
       dtype=torch.float64)
torch.Size([64, 300])
tensor([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0])
False
False


In [ ]:
 def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text_vector = batch['vector']
        # text_vector = batch['vector'].to(device)
        predictions = model(text_vector)
        
        y_pred_tag = torch.round(torch.sigmoid(predictions))
        
        # labels = batch['label'].to(device)
        labels = batch['label']
        labels = labels.float()
        labels = labels.reshape([ len(labels),1])
        loss = criterion(predictions, labels)
        
        f1 = f1_score(labels.detach().numpy(), y_pred_tag.detach().numpy())
        acc = calc_accuracy(y_pred_tag, labels)


        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text_vector = batch['vector']
            # text_vector = batch['vector'].to(device)
            predictions = model(text_vector)
            y_pred_tag = torch.round(torch.sigmoid(predictions))
            labels = batch['label']
            # labels = batch['label'].to(device)
            labels = labels.float()
            labels = labels.reshape([len(labels),1])
            loss = criterion(predictions, labels)
            f1 = f1_score(labels, y_pred_tag)
            acc = calc_accuracy(y_pred_tag, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def calc_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    # rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (preds == y).float() #convert into float for division 
    # print(correct)
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, embedding_length, output_dim):        
        super().__init__()
        
        self.embedding_length = embedding_length
        
        self.output_dim = output_dim

        # self.embedding = nn.Embedding(self.vocab_size, self.embedding_length, padding_idx=pad_idx)
        # self.embedding.weight = nn.Parameter(word_embeddings, requires_grad = False)
        self.fc1 = nn.Linear(self.embedding_length, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.4)
        
    def forward(self, text):        
        pooled_drop_op = self.dropout1(text)
        fc1_op = self.fc1(pooled_drop_op)
        fc1_drop_op = self.dropout2(fc1_op)
        fc2_op = self.fc2(fc1_drop_op)
        fc2_drop_op = self.dropout2(fc2_op) 
        fc3_op = self.fc3(fc2_drop_op)       
        return self.fc4(fc3_op)
        # text = self.relu(self.fc1(text))
        # text = self.relu(self.fc2(text))
        # text = self.relu(self.fc3(text))
        # text = self.dropout1(text)
        # return self.fc4(text)
        

In [ ]:
EMBEDDING_DIM = 300
OUTPUT_DIM = 1
model = FastText( EMBEDDING_DIM,OUTPUT_DIM)
model = model.double()

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
# criterion = criterion.to(device)

In [ ]:
N_EPOCHS =11
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, val_f1 = evaluate(model, val_dataloader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1:{train_f1}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val F1:{val_f1}')

Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.664 | Train Acc: 60.19% | Train F1:0.2222222222222222
	 Val. Loss: 0.637 |  Val. Acc: 64.63% | Val F1:0.0
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.627 | Train Acc: 66.37% | Train F1:0.7272727272727273
	 Val. Loss: 0.614 |  Val. Acc: 69.22% | Val F1:0.6666666666666667
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.604 | Train Acc: 69.57% | Train F1:0.631578947368421
	 Val. Loss: 0.595 |  Val. Acc: 71.30% | Val F1:0.5294117647058824
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.589 | Train Acc: 71.53% | Train F1:0.631578947368421
	 Val. Loss: 0.579 |  Val. Acc: 72.08% | Val F1:0.48275862068965514
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.571 | Train Acc: 72.27% | Train F1:0.7692307692307692
	 Val. Loss: 0.567 |  Val. Acc: 72.60% | Val F1:0.42857142857142855
Epoch: 06 | Epoch Time: 0m 13s
	Train Loss: 0.559 | Train Acc: 72.99% | Train F1:0.6666666666666666
	 Val. Loss: 0.556 |  Val. Acc: 73.28% | Val F1:0.6666666666666667
Epoch: 07 | E

In [ ]:

test_loss, test_acc, test_f1 = evaluate(model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1}')

Test Loss: 0.522 | Test Acc: 75.79% | Test F1: 0.5


In [ ]:
torch.save(model.state_dict(),prefix+'data/SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non Joy Reviews)Shap.pt' )

In [ ]:
model1 = FastText(EMBEDDING_DIM,OUTPUT_DIM)
model1.load_state_dict(torch.load(prefix+'data/SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non Joy Reviews)Shap.pt'))
model1.eval()

FastText(
  (fc1): Linear(in_features=300, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.4, inplace=False)
)

In [ ]:
%pip install shap
import shap

Note: you may need to restart the kernel to use updated packages.


In [ ]:
test_df = pd.read_csv(prefix+"data/new_doc_level_train.csv")
test_df.head(5)

,Unnamed: 0,message,emotion_code
0,11390,"io domani volevo vestirmi tutta figa, ma il mi...",0
1,1136,"churchhhh! then exam stuff, and making 30 dang...",1
2,9805,empty fuel tank & a $ load of traffic,1
3,16383,he who cannot forgive others destroys the brid...,1
4,12353,hoje meu dia vai ser study study and study. te...,0


In [ ]:
def get_word_embeddings(dataframe_csv, idx_limit):
  vocab={}
  bg = torch.empty((0,300))
  df = pd.read_csv(dataframe_csv)
  for idx, row in df.iterrows():
    curr_text = df.iloc[idx,1]
    curr_label = df.iloc[idx,2]
    if(curr_label==0):
      for word in word_tokenize(curr_text):
        if(word in embeddings.keys() and word not in vocab.keys()):
          vocab[word]=0
          curr_embed = torch.from_numpy(np.array(embeddings[word]))
          curr_embed = curr_embed.reshape((1, len(curr_embed)))
          bg = torch.cat((bg, curr_embed))
    if(idx==idx_limit):
      break

  return bg, vocab


In [ ]:
new_bg,_ = get_word_embeddings(prefix+"data/new_doc_level_train.csv", 1500)
print(new_bg.size())

torch.Size([3441, 300])


In [ ]:
e = shap.DeepExplainer(model, new_bg)

In [ ]:
explain_examples, vocab1  = get_word_embeddings(prefix+"data/new_doc_level_test.csv", 1200)
print(explain_examples.size())

torch.Size([2803, 300])


In [ ]:
st = time.time()
shap_vals = e.shap_values(explain_examples.float())
print(time.time()-st)

1930.1127843856812


In [ ]:
print(len(shap_vals))

2803


In [ ]:
print(len(vocab1))

2803


In [ ]:

print((shap_vals).shape)

(2803, 300)


In [ ]:
shap_means_joy = shap_vals.mean(axis=1)

In [ ]:
idx=0
for word in vocab1:
  vocab1[word]=shap_means_joy[idx]
  idx+=1

In [ ]:
for word in vocab1:
  print(word, " ", vocab1[word])

heel   -0.0005107175943770458
iets   -0.001240469941522717
hulp   -0.0009016350689279525
mag   -0.00025241699523836247
wel   2.7491713280610498e-05
toch   -0.0008157319792500586
?   -0.000774416234194835
#   -0.00014238603241273503
apparently   -0.0008656274270728479
10   0.0011033152054472097
people   -0.0002821983203273556
every   0.0007270104547079466
year   0.0011890048682039185
die   -0.000905641380816263
from   0.0001897996818315947
vending   0.0006487199865774725
machines   7.478557182896838e-05
...   6.210855533520167e-05
it   -9.178620255250907e-05
really   -0.0002084340038146039
proves   -0.00042759143052002057
how   -0.00019744056996861861
us   9.15330716742488e-05
humans   -0.0004633861052332139
can   6.332323230067318e-05
so   0.0002241473204857564
easily   -0.00038198948937284596
.   0.0003305710624527051
i   -0.00028868663864321705
hate   -0.000990568642321341
when   -0.00010751787411936728
go   6.317054821744362e-05
to   0.0004248323332331546
@   -0.00048260333373843686

In [ ]:
df = pd.DataFrame(list(vocab1.items()),columns = ['column1','column2']) 

In [ ]:
lexicon_df_joy = df.sort_values(by='column2')

In [ ]:
lexicon_df_joy.head(50)

,column1,column2
1389,reageren,-0.002391
1047,scarier,-0.002331
98,afraid,-0.002288
460,scared,-0.002179
2794,paura,-0.002000
2314,ewwwwww,-0.001985
2628,smfh,-0.001946
2617,eww,-0.001936
945,warned,-0.001917
2640,aucun,-0.001887


In [ ]:
lexicon_df_joy.tail(50)

,column1,column2
2075,prep,0.001654
1807,coffee,0.001656
1230,breakfast,0.001662
366,spend,0.001686
2238,4am,0.001703
1124,cleaning,0.001717
2106,celebratory,0.001720
1638,productivity,0.001734
1151,enjoy,0.001751
648,pray,0.001754


In [ ]:
lexicon_df_joy.to_csv(prefix+"results/SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Non Joy Reviews)Shap.csv")
